# Libraries

In [11]:
!pip install Sastrawi
!pip install clean-text
!pip install deep-translator

In [12]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from cleantext import clean
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
import sklearn

In [ ]:
# !pip freeze > requirements.txt

# Get Data from ES and prepare DF

In [14]:
import requests
import json

In [19]:
# Importing Data
url = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/data_text_classification.csv'

# Create a DataFrame
df = pd.read_csv(url,encoding='utf-8')
print(df.head())

                                      topic  \
0  Infrastruktur, Pembangunan dan Perumahan   
1  Infrastruktur, Pembangunan dan Perumahan   
2  Infrastruktur, Pembangunan dan Perumahan   
3  Infrastruktur, Pembangunan dan Perumahan   
4  Infrastruktur, Pembangunan dan Perumahan   

                                               title  \
0  Warga Yakin Pembangunan Infrastruktur di Jakar...   
1  Tahun Ini, Anjungan Pandang Tower Ulu Jembatan...   
2  Cara HK Percepat Konstruksi Tiga Gedung UPI Ba...   
3  Tol Cimaci Segera Beroperasi, Harvest City Res...   
4  Pj Gubernur Banten Al Muktabar Raih Ekbispar A...   

                                                text  
0  JAKARTA, KOMPAS.com - Warga Penjaringan, Jakar...  
1  JAKARTA, KOMPAS.com - Pemeliharaan Jembatan Am...  
2  JAKARTA, KOMPAS.com - PT Hutama Karya (Persero...  
3  JAKARTA, KOMPAS.com - Dengan tingginya angka k...  
4  KOMPAS.com - Penjabat (Pj) Gubernur Banten Al ...  


In [ ]:
len(df)

149

In [20]:
# Drop Unnamed column
df.drop(df.columns[df.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

In [21]:
df.head()

,topic,title,text
0,"Infrastruktur, Pembangunan dan Perumahan",Warga Yakin Pembangunan Infrastruktur di Jakar...,"JAKARTA, KOMPAS.com - Warga Penjaringan, Jakar..."
1,"Infrastruktur, Pembangunan dan Perumahan","Tahun Ini, Anjungan Pandang Tower Ulu Jembatan...","JAKARTA, KOMPAS.com - Pemeliharaan Jembatan Am..."
2,"Infrastruktur, Pembangunan dan Perumahan",Cara HK Percepat Konstruksi Tiga Gedung UPI Ba...,"JAKARTA, KOMPAS.com - PT Hutama Karya (Persero..."
3,"Infrastruktur, Pembangunan dan Perumahan","Tol Cimaci Segera Beroperasi, Harvest City Res...","JAKARTA, KOMPAS.com - Dengan tingginya angka k..."
4,"Infrastruktur, Pembangunan dan Perumahan",Pj Gubernur Banten Al Muktabar Raih Ekbispar A...,KOMPAS.com - Penjabat (Pj) Gubernur Banten Al ...


In [24]:
df['topic'].nunique()

8

# Preprocessing

## Delete Duplicates

In [25]:
df.duplicated(subset='title').sum()


0

In [26]:
df = df.drop_duplicates(subset='title')

## Deleting Blank (Nan) Rows

In [27]:
df = df[df['text'] != '']

In [ ]:
#row_index = 127  # Index of the row you want to print (Python indexing starts from 0)
#columns_to_print = ['title', 'text']  # List of column names to display

# Use .loc[] to select the row by its index and specify the columns to display
#row_data = df.loc[row_index, columns_to_print]

#print(row_data)

## Case Folding

In [28]:
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

In [ ]:
# Buat kolom tambahan untuk data ulasan yang telah dicasefolding
df['lwr'] = df['text'].apply(clean_lower)
df['lwr']

In [ ]:
df.head()

In [ ]:
print(df['text'][30])
print("_______________________________________________________________________________________________________________")
print(df['lwr'][30])

KABAR LUWUK  –Dinas Sosial BanggaiSegera Salurkan 2.42 MiliarProgram Gercep Gaskan Berdaya2023.Dinas Sosial Kabupaten Banggaimenerima bantuan sebesar 2.42 miliar rupiah dalam program Gerak Cepat Pengentasan Kemiskinan Berbasis Pemberdayaan Masyarakat (GERCEP GASKAN BERDAYA).Bantuan ini ditujukan untuk 242 keluarga penerima manfaat di lima kecamatan, memberikan harapan bagi warga di Kecamatan Luwuk (35 KPM), Luwuk Selatan (28 KPM), Pagimana (95 KPM), Bunta (47 KPM), dan Nuhon (37 KPM). Selasa 28/11/2023.Kepala Dinas Sosial Kabupaten Banggai, Irpan Poma melalui Kepala Bidang Fakir Miskis,Ronald Putje,menyampaikan kepada media bahwa dana bantuan ini merupakan sumbangan langsung dari Pemerintah Provinsi Sulawesi Tengah.Dinas Sosial Kabupaten Banggai bertanggung jawab dalam memfasilitasi distribusi bantuan dan ikut serta dalam pengawasan ketat agar dana tersebut benar-benar sampai kepada penerima manfaat yang tepat.Data yang tersedia menunjukkan bahwa 242 Keluarga Penerima Manfaat yang terd

## Cleansing

### Remove Numbers

In [31]:
def remove_numbers(text):
  text = re.sub('[0-9]+', '', text)
  return text

In [32]:
df['clean_number'] = df['lwr'].apply(remove_numbers)
df['clean_number']

0      jakarta, kompas.com - warga penjaringan, jakar...
1      jakarta, kompas.com - pemeliharaan jembatan am...
2      jakarta, kompas.com - pt hutama karya (persero...
3      jakarta, kompas.com - dengan tingginya angka k...
4      kompas.com - penjabat (pj) gubernur banten al ...
                             ...                        
144    kabar luwuk  –tim ii koramil -/luwukgiatpatrol...
145    kabar luwuk  –  sebuah malam yang suram meland...
146    kabar luwuk  –kejaksaan negeri banggaidan peme...
147    kabar luwuk –evaluasi penyaluran bantuan sosia...
148    kabar luwuk– pemerintah kabupaten banggaigagas...
Name: clean_number, Length: 149, dtype: object

In [33]:
df.tail()

,topic,title,text,lwr,clean_number
144,Keamanan dan Layanan Publik,Tim II Koramil 1308-01/Luwuk Giat Patroli Mala...,KABAR LUWUK –Tim II Koramil 1308-01/LuwukGiat...,kabar luwuk –tim ii koramil 1308-01/luwukgiat...,kabar luwuk –tim ii koramil -/luwukgiatpatrol...
145,Keamanan dan Layanan Publik,Kebakaran Menghanguskan Rumah Janda di Luwuk T...,KABAR LUWUK – Sebuah malam yang suram meland...,kabar luwuk – sebuah malam yang suram meland...,kabar luwuk – sebuah malam yang suram meland...
146,Ekonomi Lokal,Kejaksaan Negeri Banggai dan Pemerintah Kabupa...,KABAR LUWUK –Kejaksaan Negeri Banggaidan Peme...,kabar luwuk –kejaksaan negeri banggaidan peme...,kabar luwuk –kejaksaan negeri banggaidan peme...
147,Ekonomi Lokal,Evaluasi Penyaluran Bantuan Sosial PKH di Bang...,KABAR LUWUK –Evaluasi Penyaluran Bantuan Sosia...,kabar luwuk –evaluasi penyaluran bantuan sosia...,kabar luwuk –evaluasi penyaluran bantuan sosia...
148,Ekonomi Lokal,Pemerintah Kabupaten Banggai Gagas Visi Bangga...,KABAR LUWUK– Pemerintah Kabupaten Banggaigagas...,kabar luwuk– pemerintah kabupaten banggaigagas...,kabar luwuk– pemerintah kabupaten banggaigagas...


### Remove Punctuation

In [34]:
#Remove Puncutuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text

In [35]:
# Buat kolom tambahan untuk data description yang telah diremovepunctuation
df['clean_punct'] = df['clean_number'].apply(clean_punct)
df['clean_punct']

0      jakarta kompas com   warga penjaringan jakarta...
1      jakarta kompas com   pemeliharaan jembatan amp...
2      jakarta kompas com   pt hutama karya persero m...
3      jakarta kompas com   dengan tingginya angka ke...
4      kompas com   penjabat pj gubernur banten al mu...
                             ...                        
144    kabar luwuk   tim ii koramil  luwukgiatpatroli...
145    kabar luwuk     sebuah malam yang suram meland...
146    kabar luwuk   kejaksaan negeri banggaidan peme...
147    kabar luwuk  evaluasi penyaluran bantuan sosia...
148    kabar luwuk  pemerintah kabupaten banggaigagas...
Name: clean_punct, Length: 149, dtype: object

In [36]:
df.head()


,topic,title,text,lwr,clean_number,clean_punct
0,"Infrastruktur, Pembangunan dan Perumahan",Warga Yakin Pembangunan Infrastruktur di Jakar...,"JAKARTA, KOMPAS.com - Warga Penjaringan, Jakar...","jakarta, kompas.com - warga penjaringan, jakar...","jakarta, kompas.com - warga penjaringan, jakar...",jakarta kompas com warga penjaringan jakarta...
1,"Infrastruktur, Pembangunan dan Perumahan","Tahun Ini, Anjungan Pandang Tower Ulu Jembatan...","JAKARTA, KOMPAS.com - Pemeliharaan Jembatan Am...","jakarta, kompas.com - pemeliharaan jembatan am...","jakarta, kompas.com - pemeliharaan jembatan am...",jakarta kompas com pemeliharaan jembatan amp...
2,"Infrastruktur, Pembangunan dan Perumahan",Cara HK Percepat Konstruksi Tiga Gedung UPI Ba...,"JAKARTA, KOMPAS.com - PT Hutama Karya (Persero...","jakarta, kompas.com - pt hutama karya (persero...","jakarta, kompas.com - pt hutama karya (persero...",jakarta kompas com pt hutama karya persero m...
3,"Infrastruktur, Pembangunan dan Perumahan","Tol Cimaci Segera Beroperasi, Harvest City Res...","JAKARTA, KOMPAS.com - Dengan tingginya angka k...","jakarta, kompas.com - dengan tingginya angka k...","jakarta, kompas.com - dengan tingginya angka k...",jakarta kompas com dengan tingginya angka ke...
4,"Infrastruktur, Pembangunan dan Perumahan",Pj Gubernur Banten Al Muktabar Raih Ekbispar A...,KOMPAS.com - Penjabat (Pj) Gubernur Banten Al ...,kompas.com - penjabat (pj) gubernur banten al ...,kompas.com - penjabat (pj) gubernur banten al ...,kompas com penjabat pj gubernur banten al mu...


### Delete whitespaces

In [37]:
def normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

In [38]:
df['clean_double_ws'] = df['clean_punct'].apply(normalize_whitespace)
df['clean_double_ws']

0      jakarta kompas com warga penjaringan jakarta u...
1      jakarta kompas com pemeliharaan jembatan amper...
2      jakarta kompas com pt hutama karya persero men...
3      jakarta kompas com dengan tingginya angka kebu...
4      kompas com penjabat pj gubernur banten al mukt...
                             ...                        
144    kabar luwuk tim ii koramil luwukgiatpatroli ma...
145    kabar luwuk sebuah malam yang suram melanda de...
146    kabar luwuk kejaksaan negeri banggaidan pemeri...
147    kabar luwuk evaluasi penyaluran bantuan sosial...
148    kabar luwuk pemerintah kabupaten banggaigagas ...
Name: clean_double_ws, Length: 149, dtype: object

In [39]:
df.tail()

,topic,title,text,lwr,clean_number,clean_punct,clean_double_ws
144,Keamanan dan Layanan Publik,Tim II Koramil 1308-01/Luwuk Giat Patroli Mala...,KABAR LUWUK –Tim II Koramil 1308-01/LuwukGiat...,kabar luwuk –tim ii koramil 1308-01/luwukgiat...,kabar luwuk –tim ii koramil -/luwukgiatpatrol...,kabar luwuk tim ii koramil luwukgiatpatroli...,kabar luwuk tim ii koramil luwukgiatpatroli ma...
145,Keamanan dan Layanan Publik,Kebakaran Menghanguskan Rumah Janda di Luwuk T...,KABAR LUWUK – Sebuah malam yang suram meland...,kabar luwuk – sebuah malam yang suram meland...,kabar luwuk – sebuah malam yang suram meland...,kabar luwuk sebuah malam yang suram meland...,kabar luwuk sebuah malam yang suram melanda de...
146,Ekonomi Lokal,Kejaksaan Negeri Banggai dan Pemerintah Kabupa...,KABAR LUWUK –Kejaksaan Negeri Banggaidan Peme...,kabar luwuk –kejaksaan negeri banggaidan peme...,kabar luwuk –kejaksaan negeri banggaidan peme...,kabar luwuk kejaksaan negeri banggaidan peme...,kabar luwuk kejaksaan negeri banggaidan pemeri...
147,Ekonomi Lokal,Evaluasi Penyaluran Bantuan Sosial PKH di Bang...,KABAR LUWUK –Evaluasi Penyaluran Bantuan Sosia...,kabar luwuk –evaluasi penyaluran bantuan sosia...,kabar luwuk –evaluasi penyaluran bantuan sosia...,kabar luwuk evaluasi penyaluran bantuan sosia...,kabar luwuk evaluasi penyaluran bantuan sosial...
148,Ekonomi Lokal,Pemerintah Kabupaten Banggai Gagas Visi Bangga...,KABAR LUWUK– Pemerintah Kabupaten Banggaigagas...,kabar luwuk– pemerintah kabupaten banggaigagas...,kabar luwuk– pemerintah kabupaten banggaigagas...,kabar luwuk pemerintah kabupaten banggaigagas...,kabar luwuk pemerintah kabupaten banggaigagas ...


## Stopword Removal

### Get custom stopwords

In [40]:
url1 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-stopwords_banggai_addedwords.csv'
df_stopword1 = pd.read_csv(url1, delimiter = ";",encoding='cp1252')

In [41]:
url2 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-Stopwords(MasDevid).csv'
df_stopword2 = pd.read_csv(url2, delimiter = ";",encoding='cp1252')

In [ ]:
df_stopword.info()

In [ ]:
df_stopword

In [ ]:
df.head()

In [42]:
custom_stopword1 = list(df_stopword1['stopword'])
custom_stopword2 = list(df_stopword2['Stopwords'])

### Removal

In [ ]:
# Prepare stopword remover
#factory = StopWordRemoverFactory()
#stopwords = factory.get_stop_words() + custom_stopword
#print(stopwords)

In [43]:
# using only sastrawy
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words() + custom_stopword1 + custom_stopword2

In [44]:
len(stopwords)

946

In [45]:
dictionary = ArrayDictionary(stopwords)
stopword = StopWordRemover(dictionary)


In [ ]:
stopwords

In [46]:
def filtering(text):
  stop = stopword.remove(text)
  return stop

In [ ]:
df.head()

In [ ]:
#df.drop(['filtered'], axis=1, inplace=True)

In [47]:
df['filtered'] = df['clean_double_ws'].apply(filtering)
df['filtered']

0      warga penjaringan utara bernama josua yakin pe...
1      kompas pemeliharaan jembatan ampera program pr...
2      kompas pt hutama karya persero menerapkan tekn...
3      kompas tingginya angka kebutuhan hunian prospe...
4      com penjabat pj gubernur banten al muktabar me...
                             ...                        
144    tim ii koramil luwukgiatpatroli malam tingkatk...
145    malam suram melanda desa lauwon kecamatan timu...
146    luwuk kejaksaan negeri banggaidan pemerintah k...
147    luwuk evaluasi penyaluran bantuan sosial pkh m...
148    luwuk pemerintah banggaigagas visibanggai emas...
Name: filtered, Length: 149, dtype: object

In [ ]:
df.head()

In [ ]:
len(df)

149

In [48]:
print(df['clean_double_ws'][55])
print(df['filtered'][55])

kabar luwuk tim inafis polres banggaiselidiki kasus pencurian di desa bubung kerugian rp juta tim indonesianautomatic fingerprints identification system inafissatreskrimpolres banggaitengah intensif melakukan penyelidikan terkait kasus pencurian yang terjadi didesa bubungkecamatan luwuk selatan kejadian tersebut diketahui pada kamis sekitar pukul wita dan dilaporkan melalui nomor laporan polisi lpbxiispktres banggaipolda sulteng kamis desember menurut keterangankasat reskrim polres banggai akp tio tondytim inafis telah melakukan olah tempat kejadian perkara tkp untuk mengumpulkan bukti dan petunjuk terkait kejadian tersebut korban atas nama arifin morintoh melaporkan kejadian tersebut pada pukul wita setelah menemukan keadaan rumahnya yang telah dirusak oleh pencuri saksi saksi yang terlibat dalam kejadian ini seperti indah permata sari morintoh memberikan keterangan bahwa saat tiba di rumahnya di desa bubung ia menemukan rumah dalam keadaan berantakan ditemukannya jendela rumah bagian

## Stemming

In [49]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

In [ ]:
#factory = StemmerFactory()
#stemmer = factory.create_stemmer()

In [50]:
def stem(text):
  return stemmer.stem(text)

In [ ]:
df.head()

In [52]:
df['stemmed'] = df['filtered'].apply(stem)
df['stemmed']

0      warga jaring utara nama josua yakin bangun inf...
1      kompas pelihara jembatan ampera program priori...
2      kompas pt hutama karya persero terap teknik up...
3      kompas tinggi angka butuh huni prospek bisnis ...
4      com jabat pj gubernur banten al muktabar raih ...
                             ...                        
144    tim ii koramil luwukgiatpatroli malam tingkat ...
145    malam suram landa desa lauwon camat timur sabt...
146    luwuk jaksa negeri banggaidan perintah kabupat...
147    luwuk evaluasi salur bantu sosial pkh buka aca...
148    luwuk perintah banggaigagas visibanggai emas m...
Name: stemmed, Length: 149, dtype: object

In [ ]:
df.head()

## Export df to csv (to cutting time if we want to stepback process)

In [54]:
df.to_csv("stemmed_words_text_classification.csv", index=False)

## Importing df rom csv (cutting time if we want to stepback process)

In [ ]:
# to skip stemming process
stemmed_url= 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/stemmed_words2_tanpa_token_stopwordk2.csv'

df = pd.read_csv(stemmed_url,encoding='utf-8')

df = df.astype(str)

In [ ]:
df.head()


In [ ]:
len(df)

149

In [ ]:
print(df['filtered'][55])
print(df['stemmed'][55])

tim inafis polres banggaiselidiki pencurian desa bubung kerugian rp juta tim indonesianautomatic fingerprints identification system inafissatreskrimpolres banggaitengah intensif penyelidikan terkait pencurian terjadi didesa bubungkecamatan luwuk selatan kejadian diketahui kamis wita dilaporkan nomor polisi lpbxiispktres banggaipolda sulteng kamis desember keterangankasat reskrim polres akp tondytim inafis olah kejadian perkara tkp mengumpulkan bukti petunjuk terkait kejadian korban nama arifin morintoh melaporkan kejadian pada pukul wita menemukan rumahnya telah dirusak pencuri saksi saksi terlibat kejadian seperti indah permata sari morintoh keterangan saat rumahnya di desa bubung menemukan rumah berantakan ditemukannya jendela rumah samping dirusak cara dicungkil akp anjas suyono penjaga rumah menjelaskan pada rabu pukul wita meninggalkan rumah dalam keadaan terkunci ia terkejut indah memberitahu rumah telah sasaran pencurian akibat kejadian hilang unit keyboard merk yamaha kg kemiri

In [ ]:
print(len(df['filtered'][796]))
print(len(df['stemmed'][796]))

1167
990


## Tokenizing

In [55]:
def tokenize(text):
  tokens = nltk.tokenize.word_tokenize(text)
  return tokens

In [ ]:
df.head()

In [57]:
df['tokenized'] = df['stemmed'].apply(tokenize)
df['tokenized']

0      [warga, jaring, utara, nama, josua, yakin, ban...
1      [kompas, pelihara, jembatan, ampera, program, ...
2      [kompas, pt, hutama, karya, persero, terap, te...
3      [kompas, tinggi, angka, butuh, huni, prospek, ...
4      [com, jabat, pj, gubernur, banten, al, muktaba...
                             ...                        
144    [tim, ii, koramil, luwukgiatpatroli, malam, ti...
145    [malam, suram, landa, desa, lauwon, camat, tim...
146    [luwuk, jaksa, negeri, banggaidan, perintah, k...
147    [luwuk, evaluasi, salur, bantu, sosial, pkh, b...
148    [luwuk, perintah, banggaigagas, visibanggai, e...
Name: tokenized, Length: 149, dtype: object

In [58]:
print(df['filtered'][0])
print(df['tokenized'][0])

warga penjaringan utara bernama josua yakin pembangunan infrastruktur berjalan status kota negara dilihat infrastruktur berjalan pembangunannya berubah pembangunan tiba berhenti dia dihubungi kompas com jumat josua pembangunan infrastruktur berhenti lantaran pusat bisnis perekonomian baca status kota dicabut warga konser ikn semoga tiket pesawatnya murah kata pemerintah serta merta memfokuskan perhatian kota nusantara ikn menuturkan progres pembangunan infrastruktur bakal dibuat lambat cenderung cepat sih saya enggak akan fokus ikn doang pembangunan infrastruktur bakal tetap berjalan cukup pesat josua serupa dituturkan warga pademangan utara bernama anggita nantinya pembangunan infrastruktur lambat berarti fokus pemerintah ikn terkendala sisi kayak cuaca dan ekonomi anggita dihubungi jumat jika faktor ekonomi berkaitan pembangunan ikn anggita menyayangkannya baca pendapat warga soal hilangnya status daerah khusus kota ada sedih dan cuek jika faktor ekonomi proyek infrastruktur yang sed

In [59]:
print(len(df['stemmed'][0]))
print(len(df['tokenized'][0]))

2236
355


##Second Stopwords

In [ ]:
df.head()

In [62]:
# Define a function to remove stopwords from a list of tokens
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

# Apply the remove_stopwords function to each row in the 'tokenized' column
df['filtered2'] = df['tokenized'].apply(lambda tokens: remove_stopwords(tokens))

# Display the updated DataFrame
print(df.head())

                                      topic  \
0  Infrastruktur, Pembangunan dan Perumahan   
1  Infrastruktur, Pembangunan dan Perumahan   
2  Infrastruktur, Pembangunan dan Perumahan   
3  Infrastruktur, Pembangunan dan Perumahan   
4  Infrastruktur, Pembangunan dan Perumahan   

                                               title  \
0  Warga Yakin Pembangunan Infrastruktur di Jakar...   
1  Tahun Ini, Anjungan Pandang Tower Ulu Jembatan...   
2  Cara HK Percepat Konstruksi Tiga Gedung UPI Ba...   
3  Tol Cimaci Segera Beroperasi, Harvest City Res...   
4  Pj Gubernur Banten Al Muktabar Raih Ekbispar A...   

                                                text  \
0  JAKARTA, KOMPAS.com - Warga Penjaringan, Jakar...   
1  JAKARTA, KOMPAS.com - Pemeliharaan Jembatan Am...   
2  JAKARTA, KOMPAS.com - PT Hutama Karya (Persero...   
3  JAKARTA, KOMPAS.com - Dengan tingginya angka k...   
4  KOMPAS.com - Penjabat (Pj) Gubernur Banten Al ...   

                                           

In [63]:
#df.drop(columns=['tokenized'], inplace=True)

In [64]:
len(stopwords)

946

In [65]:
df.head()

,topic,title,text,lwr,clean_number,clean_punct,clean_double_ws,filtered,stemmed,tokenized,filtered2
0,"Infrastruktur, Pembangunan dan Perumahan",Warga Yakin Pembangunan Infrastruktur di Jakar...,"JAKARTA, KOMPAS.com - Warga Penjaringan, Jakar...","jakarta, kompas.com - warga penjaringan, jakar...","jakarta, kompas.com - warga penjaringan, jakar...",jakarta kompas com warga penjaringan jakarta...,jakarta kompas com warga penjaringan jakarta u...,warga penjaringan utara bernama josua yakin pe...,warga jaring utara nama josua yakin bangun inf...,"[warga, jaring, utara, nama, josua, yakin, ban...","[warga, jaring, utara, nama, josua, bangun, in..."
1,"Infrastruktur, Pembangunan dan Perumahan","Tahun Ini, Anjungan Pandang Tower Ulu Jembatan...","JAKARTA, KOMPAS.com - Pemeliharaan Jembatan Am...","jakarta, kompas.com - pemeliharaan jembatan am...","jakarta, kompas.com - pemeliharaan jembatan am...",jakarta kompas com pemeliharaan jembatan amp...,jakarta kompas com pemeliharaan jembatan amper...,kompas pemeliharaan jembatan ampera program pr...,kompas pelihara jembatan ampera program priori...,"[kompas, pelihara, jembatan, ampera, program, ...","[pelihara, jembatan, ampera, program, priorita..."
2,"Infrastruktur, Pembangunan dan Perumahan",Cara HK Percepat Konstruksi Tiga Gedung UPI Ba...,"JAKARTA, KOMPAS.com - PT Hutama Karya (Persero...","jakarta, kompas.com - pt hutama karya (persero...","jakarta, kompas.com - pt hutama karya (persero...",jakarta kompas com pt hutama karya persero m...,jakarta kompas com pt hutama karya persero men...,kompas pt hutama karya persero menerapkan tekn...,kompas pt hutama karya persero terap teknik up...,"[kompas, pt, hutama, karya, persero, terap, te...","[pt, hutama, karya, persero, terap, teknik, up..."
3,"Infrastruktur, Pembangunan dan Perumahan","Tol Cimaci Segera Beroperasi, Harvest City Res...","JAKARTA, KOMPAS.com - Dengan tingginya angka k...","jakarta, kompas.com - dengan tingginya angka k...","jakarta, kompas.com - dengan tingginya angka k...",jakarta kompas com dengan tingginya angka ke...,jakarta kompas com dengan tingginya angka kebu...,kompas tingginya angka kebutuhan hunian prospe...,kompas tinggi angka butuh huni prospek bisnis ...,"[kompas, tinggi, angka, butuh, huni, prospek, ...","[angka, butuh, huni, prospek, bisnis, properti..."
4,"Infrastruktur, Pembangunan dan Perumahan",Pj Gubernur Banten Al Muktabar Raih Ekbispar A...,KOMPAS.com - Penjabat (Pj) Gubernur Banten Al ...,kompas.com - penjabat (pj) gubernur banten al ...,kompas.com - penjabat (pj) gubernur banten al ...,kompas com penjabat pj gubernur banten al mu...,kompas com penjabat pj gubernur banten al mukt...,com penjabat pj gubernur banten al muktabar me...,com jabat pj gubernur banten al muktabar raih ...,"[com, jabat, pj, gubernur, banten, al, muktaba...","[jabat, pj, gubernur, banten, al, muktabar, ra..."


In [66]:
print(df['tokenized'][55])
print(df['filtered2'][55])

['tim', 'inafis', 'polres', 'banggaiselidiki', 'curi', 'desa', 'bubung', 'rugi', 'rp', 'juta', 'tim', 'indonesianautomatic', 'fingerprints', 'identification', 'system', 'inafissatreskrimpolres', 'banggaitengah', 'intensif', 'lidi', 'kait', 'curi', 'jadi', 'desa', 'bubungkecamatan', 'luwuk', 'selatan', 'jadi', 'tahu', 'kamis', 'wita', 'lapor', 'nomor', 'polisi', 'lpbxiispktres', 'banggaipolda', 'sulteng', 'kamis', 'desember', 'keterangankasat', 'reskrim', 'polres', 'akp', 'tondytim', 'inafis', 'olah', 'jadi', 'perkara', 'tkp', 'kumpul', 'bukti', 'tunjuk', 'kait', 'jadi', 'korban', 'nama', 'arifin', 'morintoh', 'lapor', 'jadi', 'pada', 'pukul', 'wita', 'temu', 'rumah', 'telah', 'rusak', 'curi', 'saksi', 'saksi', 'libat', 'jadi', 'seperti', 'indah', 'permata', 'sari', 'morintoh', 'terang', 'saat', 'rumah', 'di', 'desa', 'bubung', 'temu', 'rumah', 'beranta', 'temu', 'jendela', 'rumah', 'samping', 'rusak', 'cara', 'cungkil', 'akp', 'anjas', 'suyono', 'jaga', 'rumah', 'jelas', 'pada', 'rabu'

# Text Classification

In [ ]:
# For manual Review
#import random

# Set the number of random samples you want to select
#num_samples = 100  # Adjust this number as needed

# Select random samples from the DataFrame
#random_indices = random.sample(range(len(df)), num_samples)
#random_data = df.iloc[random_indices][["title", "text"]]

# Save random data to a CSV file
#random_data.to_csv("random_data.csv")


In [ ]:
df.info()

## Preprocessing (Text Classification)

In [ ]:
df.head()

In [71]:
# Spliting database to train_set and test_set
from sklearn.model_selection import train_test_split

X = df['filtered2']
y = df['topic']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)

In [72]:
X_train.head()

95     [lembaga, jamin, mutu, kembang, didik, lpmpp, ...
104    [program, studi, prodi, manajemen, fakultas, e...
66     [juni, inflasi, year, on, year, yoy, kota, cat...
0      [warga, jaring, utara, nama, josua, bangun, in...
141    [anggota, satlantaspolres, banggaimelaksanakan...
Name: filtered2, dtype: object

In [ ]:
print(y_test.unique())


#df['Topic'].nunique()

In [74]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

# Fit Vectorizer to the data (build a vocab, count the number of words..)
# and transfrom the original text message to ---> Vector

from itertools import chain

# Assuming X_train is a list of lists or a list of tokens
flattened_X_train = list(chain.from_iterable(X_train))
flattened_X_test = list(chain.from_iterable(X_test))

# Assuming X_train is a list of tokens
X_train = [' '.join(tokens) for tokens in X_train]
X_test = [' '.join(tokens) for tokens in X_test]

X_train_counts = count_vect.fit_transform(X_train)


In [75]:
X_train_counts

<99x3291 sparse matrix of type '<class 'numpy.int64'>'
	with 8942 stored elements in Compressed Sparse Row format>

In [ ]:
X_train.shape

(99,)

In [76]:
X_train_counts.shape

(99, 3291)

In [77]:
# TFIDFVectorizer and LinearSVC with pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('tfidf', TfidfVectorizer()),('clf',LinearSVC())])
text_clf.fit(X_train, y_train)



Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [78]:
predictions = text_clf.predict(X_test)

In [79]:
# Checking performance using confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(f"Accuracy {metrics.accuracy_score(y_test, predictions)}")

[[ 4  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0]
 [ 0  0 16  0  1  1  0  0]
 [ 1  0  0  4  0  0  0  0]
 [ 3  0  2  0  5  1  0  0]
 [ 0  0  0  0  0  4  0  0]
 [ 0  0  2  0  0  0  0  0]
 [ 0  2  1  0  0  0  0  0]]
                                          precision    recall  f1-score   support

                           Ekonomi Lokal       0.50      1.00      0.67         4
Infrastruktur, Pembangunan dan Perumahan       0.60      1.00      0.75         3
             Keamanan dan Layanan Publik       0.76      0.89      0.82        18
                        Kesehatan Publik       1.00      0.80      0.89         5
        Lingkungan dan Partisipasi Warga       0.83      0.45      0.59        11
                              Pendidikan       0.67      1.00      0.80         4
                                 Politik       0.00      0.00      0.00         2
                            Transportasi       0.00      0.00      0.00         3

                                accuracy   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Input Data

In [81]:
# Input
text_url = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/input_text.csv'

# Read CSV file with 'latin-1' encoding
input_df = pd.read_csv(text_url, encoding='utf-8')

# Convert all columns to string
input_df = input_df.astype(str)

In [ ]:
input_df.head()

## Preprocessing

In [ ]:
# Delete Duplicates
input_df.duplicated(subset='title').sum()
input_df = input_df.drop_duplicates(subset='title')

# Delete Blank text (Nan) Rows
input_df = input_df[input_df['text'] != '']

# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

input_df['lwr'] = input_df['text'].apply(clean_lower)
input_df['lwr']


# Remove Numbers
def remove_numbers(text):
  text = re.sub('[0-9]+', '', text)
  return text

input_df['clean_number'] = input_df['lwr'].apply(remove_numbers)
input_df['clean_number']


# Remove Punctuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text

input_df['clean_punct'] = input_df['clean_number'].apply(clean_punct)
input_df['clean_punct']


# Delete Whitespaces
def normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

input_df['clean_double_ws'] = input_df['clean_punct'].apply(normalize_whitespace)
input_df['clean_double_ws']


In [ ]:
len(input_df)

In [ ]:
input_df.head()

In [92]:
# Adding Stopwords
url1 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-stopwords_banggai_addedwords.csv'
df_stopword1 = pd.read_csv(url1, delimiter = ";",encoding='cp1252')

url2 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-Stopwords(MasDevid).csv'
df_stopword2 = pd.read_csv(url2, delimiter = ";",encoding='cp1252')

custom_stopword1 = list(df_stopword1['stopword'])
custom_stopword2 = list(df_stopword2['Stopwords'])

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words() + custom_stopword1 + custom_stopword2


# 1st Stopwords Removal
def filtering(text):
  stop = stopword.remove(text)
  return stop

input_df['filtered'] = input_df['clean_double_ws'].apply(filtering)
input_df['filtered']


# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

def stem(text):
  return stemmer.stem(text)

input_df['stemmed'] = input_df['filtered'].apply(stem)
input_df['stemmed']


# Tokenizing
def tokenize(text):
  tokens = nltk.tokenize.word_tokenize(text)
  return tokens

input_df['tokenized'] = input_df['stemmed'].apply(tokenize)
input_df['tokenized']


# 2nd Stopwords Removal
# Define a function to remove stopwords from a list of tokens
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

# Apply the remove_stopwords function to each row in the 'tokenized' column
input_df['filtered2'] = input_df['tokenized'].apply(lambda tokens: remove_stopwords(tokens))

In [ ]:
input_df.head()

In [98]:
#input_df.drop(columns=['text2'], inplace=True)

In [100]:
# Convert tokenized text to string representation
#text_strings = [' '.join(doc) for doc in df['tokenized']]
input_df['text2'] = input_df['tokenized'].apply(lambda x: ' '.join(x))


In [101]:
input_df.head()

,title,text,lwr,clean_number,clean_punct,clean_double_ws,filtered,stemmed,tokenized,filtered2,text2
0,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi di ...,luwuk suasana pasar murah bersubsidi semarak k...,luwuk suasana pasar murah subsidi semarak hadi...,"[luwuk, suasana, pasar, murah, subsidi, semara...","[suasana, pasar, murah, subsidi, semarak, hadi...",luwuk suasana pasar murah subsidi semarak hadi...
1,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati...","kabar luwuk – film horor“piara mayat”,kreati...","kabar luwuk – film horor“piara mayat”,kreati...",kabar luwuk film horor piara mayat kreativ...,kabar luwuk film horor piara mayat kreativitas...,luwuk film horor piara mayat kreativitasanak m...,luwuk film horor piara mayat kreativitasanak m...,"[luwuk, film, horor, piara, mayat, kreativitas...","[film, horor, piara, mayat, kreativitasanak, m...",luwuk film horor piara mayat kreativitasanak m...
2,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...,kabar luwuk –festival malabot tumbe 2023kemba...,kabar luwuk –festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali mer...,luwuk festival malabot tumbe meriahkankabupate...,luwuk festival malabot tumbe meriahkankabupate...,"[luwuk, festival, malabot, tumbe, meriahkankab...","[festival, malabot, tumbe, meriahkankabupaten,...",luwuk festival malabot tumbe meriahkankabupate...
3,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan ...,kerja cepat terkoordinir penerbitan sttp luwuk...,kerja cepat terkoordinir terbit sttp luwuk rak...,"[kerja, cepat, terkoordinir, terbit, sttp, luw...","[kerja, cepat, terkoordinir, terbit, sttp, rak...",kerja cepat terkoordinir terbit sttp luwuk rak...
4,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinasi ...,luwuk bawaslu banggaiperkuat koordinasi dengan...,luwuk bawaslu banggaiperkuat koordinasi dengan...,"[luwuk, bawaslu, banggaiperkuat, koordinasi, d...","[bawaslu, banggaiperkuat, koordinasi, denganpa...",luwuk bawaslu banggaiperkuat koordinasi dengan...


# Predicting Input Data

In [104]:

# Assuming 'selected_features' is a list of feature names used during model training
X_input = input_df['text2']

# 3. Predict using the model
predictions = text_clf.predict(X_input)

# 4. Add predictions to a new column in the DataFrame
input_df['predictions'] = predictions

In [105]:
input_df.head()

,title,text,lwr,clean_number,clean_punct,clean_double_ws,filtered,stemmed,tokenized,filtered2,text2,predictions
0,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi di ...,luwuk suasana pasar murah bersubsidi semarak k...,luwuk suasana pasar murah subsidi semarak hadi...,"[luwuk, suasana, pasar, murah, subsidi, semara...","[suasana, pasar, murah, subsidi, semarak, hadi...",luwuk suasana pasar murah subsidi semarak hadi...,Keamanan dan Layanan Publik
1,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati...","kabar luwuk – film horor“piara mayat”,kreati...","kabar luwuk – film horor“piara mayat”,kreati...",kabar luwuk film horor piara mayat kreativ...,kabar luwuk film horor piara mayat kreativitas...,luwuk film horor piara mayat kreativitasanak m...,luwuk film horor piara mayat kreativitasanak m...,"[luwuk, film, horor, piara, mayat, kreativitas...","[film, horor, piara, mayat, kreativitasanak, m...",luwuk film horor piara mayat kreativitasanak m...,Pendidikan
2,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...,kabar luwuk –festival malabot tumbe 2023kemba...,kabar luwuk –festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali mer...,luwuk festival malabot tumbe meriahkankabupate...,luwuk festival malabot tumbe meriahkankabupate...,"[luwuk, festival, malabot, tumbe, meriahkankab...","[festival, malabot, tumbe, meriahkankabupaten,...",luwuk festival malabot tumbe meriahkankabupate...,Lingkungan dan Partisipasi Warga
3,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan ...,kerja cepat terkoordinir penerbitan sttp luwuk...,kerja cepat terkoordinir terbit sttp luwuk rak...,"[kerja, cepat, terkoordinir, terbit, sttp, luw...","[kerja, cepat, terkoordinir, terbit, sttp, rak...",kerja cepat terkoordinir terbit sttp luwuk rak...,Keamanan dan Layanan Publik
4,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinasi ...,luwuk bawaslu banggaiperkuat koordinasi dengan...,luwuk bawaslu banggaiperkuat koordinasi dengan...,"[luwuk, bawaslu, banggaiperkuat, koordinasi, d...","[bawaslu, banggaiperkuat, koordinasi, denganpa...",luwuk bawaslu banggaiperkuat koordinasi dengan...,Keamanan dan Layanan Publik


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#cv = CountVectorizer(max_df=0.9, min_df=2, stop_words= stopwords)

vect =TfidfVectorizer(max_df=0.85, min_df=2, stop_words= stopwords)

dtm = vect.fit_transform(text_strings)

In [ ]:
dtm

<966x5126 sparse matrix of type '<class 'numpy.float64'>'
	with 77134 stored elements in Compressed Sparse Row format>

In [ ]:
dtm

<975x5342 sparse matrix of type '<class 'numpy.float64'>'
	with 102414 stored elements in Compressed Sparse Row format>

# Miscelenius

## Delete Most and Least Appeared Words. Convert it to DTM





## Convert df to excel

In [ ]:
# Assuming df1 is the DataFrame where you want to add the "title" column,
# and df2 is the DataFrame containing the "text" and "title" columns to match against

import pandas as pd

# Iterate through rows of df1
for index, row in input_df.iterrows():
    # Iterate through rows of df2
    for index2, row2 in df.iterrows():
        # Check if the "text" values match
        if row['text'] == row2['text']:
            # Assign the corresponding "title" value from df2 to df1
            input_df.at[index, 'title'] = row2['title']
            # Break the inner loop as we found a match
            break

# Display df1 with the added "title" column
print(input_df())

                                                text     topics_name  \
0  KABAR LUWUK –  Suasana Pasar Murah bersubsidi ...  Layanan Publik   
1  KABAR LUWUK  –  Film Horor“Piara Mayat”,Kreati...  Layanan Publik   
2  KABAR LUWUK  –Festival Malabot Tumbe 2023Kemba...  Layanan Publik   
3  “Kerja Cepat dan Terkoordinir dalam Penerbitan...  Layanan Publik   
4  KABAR LUWUK  –Bawaslu BanggaiPerkuat Koordinas...  Layanan Publik   

                                               title  
0  Mobil Sim Keliling Polres Banggai Meriahkan Pa...  
1  Film Horor “Piara Mayat”,  Kreativitas Anak Mu...  
2  Festival Malabot Tumbe 2023 Kembali Meriahkan ...  
3  Rakor Kampanye Damai Pemilu 2024, Polres Bangg...  
4  Bawaslu Banggai Perkuat Koordinasi dengan Panw...  


In [ ]:
df.to_excel('lexicon_analytics_banggai_news.xlsx', index=False)